In [2]:
import pandas as pd
import numpy as np

from scipy.stats import gmean
from scipy.optimize import root_scalar

from expon_mixture import ExponMixture 

In [3]:
df = pd.read_csv("all_data.csv", index_col=0)

In [7]:
names = []
data = []
# Iterate over all rows
for index, row in df.iterrows():
    mean = np.mean(row[1:]) # For the computation of the speedup, the mean is needed
    cumsum = 0.0 # This is used for the conditional expectation
    current_min_exp = np.inf
    optimal_t = 0.0
    for i in range(1, 301): # The first column is the instance type. This is irrelevant here.
        t = row[i] 
        cumsum += float(t) # Updata the partial expectation
        ecdf = i/300.0 # We measured 300 runs
        geom_part = (1.0 - ecdf)/ecdf * t # This part describes the geometric part of the expectation with restarts
        cond_exp = cumsum / i # partial expectation -> conditional expectation
        if geom_part + cond_exp < current_min_exp: # Update the best restart time if necessary.
            current_min_exp = geom_part + cond_exp
            optimal_t = t
    names.append(index)
    data.append([row[0], mean, current_min_exp, mean/current_min_exp, optimal_t])

optimal_df = pd.DataFrame(data, index=names, columns=["type", "expectation no restarts" ,"expectation with best restarts", "speedup", "best restart time"])

In [8]:
optimal_df

,type,expectation no restarts,expectation with best restarts,speedup,best restart time
gen_n210_m903_k3SAT_seed2473397791.cnf,barthel,3.214107e+03,3.214107e+03,1.000000,1.293200e+04
gen_n210_m903_k3SAT_seed862748622.cnf,barthel,1.342518e+04,1.308250e+04,1.026194,1.180100e+04
gen_n210_m903_k3SAT_seed4006075830.cnf,barthel,1.295157e+03,1.293375e+03,1.001378,5.594000e+03
gen_n210_m903_k3SAT_seed1547818438.cnf,barthel,1.365623e+03,1.363372e+03,1.001652,4.191000e+03
gen_n210_m903_k3SAT_seed3919912883.cnf,barthel,1.511253e+03,1.500828e+03,1.006947,4.381000e+03
...,...,...,...,...,...
gen_n70_m385_k3SAT_seed2030441879.cnf,qhid,3.993567e+02,3.990936e+02,1.000659,1.710000e+03
gen_n70_m385_k3SAT_seed3717411169.cnf,qhid,2.097433e+02,2.097433e+02,1.000000,8.900000e+02
gen_n70_m385_k3SAT_seed684617509.cnf,qhid,2.370867e+02,2.305567e+02,1.028322,5.280000e+02
gen_n70_m385_k3SAT_seed1293934752.cnf,qhid,1.008646e+11,8.186059e+10,1.232151,4.239737e+10


# Calculate the average speedup over all instances

In [9]:
gmean(optimal_df['speedup'])

18.19553445091588

# Calculate the average speedup over the barthel instances

In [10]:
df1 = optimal_df[optimal_df['type'] == 'barthel']
gmean(df1['speedup'])

1.0984796379053687

# Calculate the average speedup over the komb instances

In [11]:
df1 = optimal_df[optimal_df['type'] == 'komb']
gmean(df1['speedup'])

156.9581117219696

# Calculate the average speedup over the qhid instances

In [12]:
df1 = optimal_df[optimal_df['type'] == 'qhid']
gmean(df1['speedup'])

34.93965989136677

# Calculate the average speedup over the qhid and komb instances

In [13]:
df1 = optimal_df[(optimal_df['type'] == 'qhid') | (optimal_df['type'] == 'komb')]
gmean(df1['speedup'])

74.05445996533072

# Calculate the average speedup on instances with two components

In [14]:
fits = pd.read_csv('../fits/expon_mix_2comp_fits_mod_name.txt', index_col=0)

In [15]:
df1 = optimal_df[fits['p1']<1.0]
gmean(df1['speedup'])

228.97264635150276

## Add the number of variables to the fit

In [16]:
features = pd.read_csv('../calculate_features/features_train_mod_name.csv')
features.set_index('instance', inplace=True)
fits['n'] = features[['nvarsOrig']]

In [17]:
fits.head()

,p1,p2,lambda1,lambda2,n
instance,,,,,
gen_n210_m903_k3SAT_seed2473397791.cnf,1.0,NaN,3214.106667,NaN,210.0
gen_n210_m903_k3SAT_seed862748622.cnf,1.0,NaN,13425.176667,NaN,210.0
gen_n210_m903_k3SAT_seed4006075830.cnf,1.0,NaN,1295.156667,NaN,210.0
gen_n210_m903_k3SAT_seed1547818438.cnf,1.0,NaN,1365.623333,NaN,210.0
gen_n210_m903_k3SAT_seed3919912883.cnf,1.0,NaN,1511.253333,NaN,210.0


## Setup functions for numerical optimization

In [18]:
def condition(t, rv, b):
    if t <= 0:
        return -np.inf
    F = rv.cdf(t)
    result = (F - 1.0)*t
    result += F*(1-F)/(rv.pdf(t))
    result -= rv.partial_exp(t)
    return result - b

In [19]:
def find_restart_time(p1, lambda1, lambda2, n, c=1.5):
    ps = [p1, 1.0-p1]
    scales = [lambda1, lambda2]
    rv = ExponMixture(ps, scales)
    b = c*n
    solution = root_scalar(condition, args=(rv, b), x0=10.0*b, x1=b, method='secant', xtol=1.0)
    return np.ceil(solution.root + b)

In [20]:
def evaluate_restart_time(data, t):
    mean = np.mean(data) # For the computation of the speedup, the mean is needed
    cumsum = 0.0 # This is used for the conditional expectation
    i = 0
    while data[i] <= t:
        cumsum += data[i]
        i += 1
    if i == 0:
        return 0
    ecdf = i/300.0 # We measured 300 runs
    geom_part = (1.0 - ecdf)/ecdf * t # This part describes the geometric part of the expectation with restarts
    cond_exp = cumsum/float(i) # partial expectation -> conditional expectation
    speedup = mean/(geom_part + cond_exp)
    return speedup

## Calculate speedups on true mixtures

In [31]:
mixed_fits = fits[fits['p1']<1.0]
underestimate = 0
speedups = []
for index, row in mixed_fits.iterrows():
    restart_time = find_restart_time(row[0], row[2], row[3], row[4], c=1.5)
    speedup = evaluate_restart_time(np.asarray(df.loc[index])[1:],  restart_time)
    if speedup == 0:
        underestimate += 1
#         print(index, "lamba1", row[2], "lambda2", row[3], "restart time:", restart_time)
    else:
        print(speedup)
        speedups.append(speedup)

1.0297045668398577
1.002513935390249
1.1278374866172416
1.0080260263871315
1.0550916283813656
1.6240404654365284
1.0362653738363266
1.0170228369960141
4.079519536707226
1.0606755259057308
1.0406679110189865
1.049926997029503
0.8865885610633896
1.030593615177877
1.0793516449802116
107900.79292023435
9382.815243725503
1855847.2119447782
8.071205669328503
16.92435188949622
5289.386835150405
245.78003383143852
9.414259079976077
148.03102840415906
55933.505160439556
46245.8475305699
13.615590216334116
6.45254851517665
11299.055871820661
12.73735212925223
12.256130910229713
63512.927006238875
6.850333991365335
1.7890328657100358
173.94327216746223
379.0299943901607
107.99765368177644
327.412648520895
67.75177254225619
81.59516780205298
1705.4925838536305
12.877389872852262
1110.0546750681647
110.58047715392301
6249.48651547729
78.1799300563144
25.45336185931454
188.82925010489376
34949.53244293309
96.11912721442204
1.9810373934837182
3.954793197740398
1046.4821621232607
17.118582964806194
16

## Calculate average speedup

In [32]:
gmean(speedups)

157.35845337386866

In [34]:
print(f"{underestimate} of {len(mixed_fits)}")

5 of 158


# Average speedup on all instances

In [30]:
underestimate = 0
speedups = []
for index, row in fits.iterrows():
    if row[0] < 1.0:
        restart_time = find_restart_time(row[0], row[2], row[3], row[4], c=1.5)
        speedup = evaluate_restart_time(np.asarray(df.loc[index])[1:],  restart_time)
        if speedup == 0:
            underestimate += 1
        else:
            speedups.append(speedup)
    else:
        speedups.append(1.0) # In this case restarts are not performed, therefore no speedup or slowdown can be observed

In [25]:
print(speedups)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0297045668398577, 1.002513935390249, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.1278374866172416, 1.0, 1.0080260263871315, 1.0550916283813656, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.6240404654365284, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0362653738363266, 1.0, 1.0170228369960141, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.079519536707226, 1.0, 1.0, 1.0, 1.0, 1.0606755259057308, 1.0, 1.0406679110189865, 1.0, 1.0, 1.049926997029503, 1.0, 1.0, 0.8865885610633896, 1.0, 1.030593615177877, 1.0793516449802116, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 107900.79292023435, 9382.815243725503, 1855847.2119447782, 8.071205669328503, 16.92435188949622, 5289.386835150405, 245.78003383143852, 1.0, 9.414259079976077, 148.03102840415906, 55933.505160439556, 46245.8475305699, 13.615590216334116, 6.45254851517665, 11299.055871820661, 12

In [26]:
gmean(speedups)

13.784912277825212

In [27]:
underestimate

5

## Average speedup on all qhid and komb instances

In [37]:
underestimate = 0
speedups = []
for index, row in fits.iterrows():
    if optimal_df['type'].loc[index] == 'barthel':
        continue
    if row[0] < 1.0:
        restart_time = find_restart_time(row[0], row[2], row[3], row[4], c=1.5)
        speedup = evaluate_restart_time(np.asarray(df.loc[index])[1:],  restart_time)
        if speedup == 0:
            underestimate += 1
        else:
            speedups.append(speedup)
    else:
        speedups.append(1.0) # In this case restarts are not performed, therefore no speedup or slowdown can be observed

In [39]:
gmean(speedups)

52.313283821216174

In [40]:
print(f"{underestimate} of {len(speedups)}")

5 of 195


## Average speedup on qhid and komb instances with two components

In [41]:
underestimate = 0
speedups = []
for index, row in fits.iterrows():
    if optimal_df['type'].loc[index] == 'barthel':
        continue
    if row[0] < 1.0:
        restart_time = find_restart_time(row[0], row[2], row[3], row[4], c=1.5)
        speedup = evaluate_restart_time(np.asarray(df.loc[index])[1:],  restart_time)
        if speedup == 0:
            underestimate += 1
        else:
            speedups.append(speedup)

In [42]:
gmean(speedups)

268.20906585871154

In [43]:
print(f"{underestimate} of {len(speedups)}")

5 of 138


# Restriction to qhid instances with 50 variables, komb instances with 100 variables and instances with 'true' mixtures.

In [47]:
underestimate = 0
speedups = []
for index, row in fits.iterrows():
    if optimal_df['type'].loc[index] == 'barthel':
        continue
    if (not 'n100' in index) and (not 'n50' in index):
        continue
    if row[0] < 1.0:
        restart_time = find_restart_time(row[0], row[2], row[3], row[4], c=1.5)
        speedup = evaluate_restart_time(np.asarray(df.loc[index])[1:],  restart_time)
        if speedup == 0:
            underestimate += 1
        else:
            speedups.append(speedup)

In [48]:
speedups

[620.1968218182466,
 44815.78034950395,
 123.34368652137115,
 420.2655348382592,
 6510.379501330928,
 11.89048281990985,
 3935.0425632939405,
 2963.0738583978755,
 2813.7313690752967,
 131.76070035379172,
 144.38088863098338,
 43.95741329626945,
 1545.7240615850437,
 10.341073559555944,
 825.373705273878,
 214.20243612802926,
 934.4153161283779,
 3.124183428399881,
 123904.85964563688,
 1577.9025154259334,
 109.72823807316158,
 88.66375559150092,
 723.1768321481283,
 126544.23691132113,
 56.91850707371557,
 20.65459508818311,
 108.2899933386873,
 32.17609205935221,
 2933.1527167604154,
 1965.5430259269956]

In [49]:
gmean(speedups)

464.7323565468142